# Asset Pricing Models (APM)

We will have a look at the asset pricing models (Capital Asset Pricing Model **CAPM**, Fama-French 3 Factor **FF3**, Fama-French 5 Factor **FF5**), and how to run these models in Python. 

**Model regression framework (CAPM):** 

$$
r_{}-rf = \alpha + \beta_{} (\overline{r_{m}}-rf) + \epsilon
$$

**Model regression framework (FF3):**

$$
r_{}-rf = \alpha + \beta_{}(\overline{r_{m}}-rf) + \beta_{s}SMB + \beta_{v}HML + \epsilon
$$

**Model regression framework (FF5):**

$$
r_{}-rf = \alpha + \beta_{}(\overline{r_{m}}-rf) + \beta_{s}SMB + \beta_{v}HML + \beta_{p}RMW + \beta_{i}CMA +  \epsilon
$$

SMB: The return spread of small minus large stocks (size).

HML: The return of cheap minus expensive stocks (value).

RMW: The return spread of the most profitable firms minus the least profitable (profit).

CMA: The return spread of firms that invest conservatively minus aggressively (investment).

In [2]:
import sys
sys.path.insert(0, '../scripts/')
from database import Stock
import utilities as ut
from utilities import clean_ticker
from utilities import clean_tickers
from utilities import to_return
import pandas as pd
import numpy as np 
import scipy as sp
import statsmodels.formula.api as sm 
from statsmodels.iolib.summary2 import summary_col
import datetime as dt
from datetime import datetime
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import matplotlib
from pandas.tseries.offsets import BDay
end = pd.datetime.today().date()
start = end - 252 * BDay() * 20
import warnings
warnings.filterwarnings("ignore")
plt.style.use('dark_background')

In [3]:
def get_adjusted_ohlc(sym):
    df = Stock(sym, start=start, end=end).df
    df.index = pd.to_datetime(df.index)
    ohlc = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    #ohlc.reset_index(inplace=True)
    return ohlc

In [4]:
symbols = ['TTWO', '7CD.F', 'GME', 'ATVI', 'CCOEY', 'TCEHY', 'EA']
get_px = lambda x: get_adjusted_ohlc(x)['Close']
stocks = pd.DataFrame({sym:get_px(sym) for sym in symbols}).dropna()

In [5]:
def asset_pricing_models(df_stk):
    stkName = df_stk.name
    import pandas_datareader.data as web 
    df_factors = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench')[0]
    df_factors.rename(columns={'Mkt-RF': 'MKT'}, inplace=True)
    df_factors['MKT'] = df_factors['MKT']/100
    df_factors['SMB'] = df_factors['SMB']/100
    df_factors['HML'] = df_factors['HML']/100
    df_factors['RMW'] = df_factors['RMW']/100
    df_factors['CMA'] = df_factors['CMA']/100
    df_stock_factor = pd.merge(df_stk,df_factors,left_index=True,right_index=True)
    df_stock_factor['XsRet'] = df_stock_factor[stkName] - df_stock_factor['RF'] 
    
    CAPM = sm.ols(formula = 'XsRet ~ MKT', data=df_stock_factor).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    FF3 = sm.ols( formula = 'XsRet ~ MKT + SMB + HML', data=df_stock_factor).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    FF5 = sm.ols( formula = 'XsRet ~ MKT + SMB + HML + RMW + CMA', data=df_stock_factor).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    
    CAPMtstat = CAPM.tvalues
    FF3tstat = FF3.tvalues
    FF5tstat = FF5.tvalues
    
    CAPMcoeff = CAPM.params
    FF3coeff = FF3.params
    FF5coeff = FF5.params
    
    CAPMpredi = CAPM.predict()
    FF3predi = FF3.predict()
    FF5predi = FF5.predict()
    
    CAPMresid = CAPM.resid
    FF3resid = FF3.resid
    FF5resid = FF5.resid

    results_df = pd.DataFrame({'CAPMcoeff':CAPMcoeff,'CAPMtstat':CAPMtstat,
                               'FF3coeff':FF3coeff, 'FF3tstat':FF3tstat,
                               'FF5coeff':FF5coeff, 'FF5tstat':FF5tstat},
    index = ['Intercept', 'MKT', 'SMB', 'HML', 'RMW', 'CMA'])
    
    df_stock_factor['CAPM'] = CAPMpredi
    df_stock_factor['CAPMResiduals'] = CAPMresid
    # Fama-French 3 Factor Model (FF3)
    df_stock_factor['FF3'] = FF3predi
    df_stock_factor['FF3Residuals'] = FF3resid 
    # Fama-French 5 Factor Model (FF5)
    df_stock_factor['FF5'] = FF5predi
    df_stock_factor['FF5Residuals'] = FF5resid
    dfoutput = summary_col([CAPM, FF3, FF5],stars=True,float_format='%0.4f',
                  model_names=['CAPM','FF3','FF5'],
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'Adjusted R2':lambda x: "{:.4f}".format(x.rsquared_adj)}, 
                             regressor_order = ['Intercept', 'MKT', 'SMB', 'HML', 'RMW', 'CMA'])
    
    results_df.to_excel('../data/factor_data/regression_stats/stats_'+stkName+'.xlsx')
    df_stock_factor.to_excel('../data/factor_data/stock_factor/'+stkName+'.xlsx')

    return print(dfoutput)

In [6]:
for sym in stocks:
    data = to_return(stocks[sym],ret='simple')
    data.dropna(inplace=True)
    data.name=sym
    print('===============================================')
    print(sym+' - asset pricing models & factor regression')
    print('===============================================')
    asset_pricing_models(data)
    print('                                               ')

TTWO - asset pricing models & factor regression

                  CAPM       FF3        FF5    
-----------------------------------------------
Intercept      -0.0048*** -0.0054*** -0.0053***
               (0.0009)   (0.0008)   (0.0008)  
MKT            0.7888***  0.8840***  0.8692*** 
               (0.0722)   (0.0635)   (0.0687)  
SMB                       0.1755     0.1407    
                          (0.1475)   (0.1557)  
HML                       -0.9093*** -0.8402***
                          (0.1074)   (0.1287)  
RMW                                  -0.0771   
                                     (0.2183)  
CMA                                  -0.2241   
                                     (0.2512)  
R-squared      0.2252     0.3678     0.3690    
R-squared Adj. 0.2240     0.3648     0.3641    
N              643        643        643       
Adjusted R2    0.2240     0.3648     0.3641    
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01
                            

For all x3 models, it shows that the market factor (MKT) is statistically significant and positive, which means that the stocks have a significant exposure to market risk. Even for the FF3 and FF5, when we allow other risk factors into the model, the market risk factor remains significant (as given by the ***).